# Installing and importing packages

In [1]:
sc.install_pypi_package("boto3")
sc.install_pypi_package("pandas==1.0.3")
sc.install_pypi_package("scipy==1.4.1")
sc.install_pypi_package("matplotlib==3.2.1")
sc.install_pypi_package("seaborn==0.11.0")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1654031629754_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
import boto3
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pyspark.ml import Pipeline
import pyspark.sql.functions as f
import pyspark.sql.types as t
from sparknlp.base import *
from sparknlp.annotator import *
import sparknlp
import ast

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read data from S3

In [66]:
data = spark.read.option("header", True).option("multiline", True).option("escape","\"").csv('s3://lsc-projct/*.csv')
data = data.na.drop(subset=['username', 'content'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
# convert columns to their original types
cols = ['reply', 'retweet', 'likes']
for col_name in cols:
    data = data.withColumn(col_name, f.col(col_name).cast('int'))

data = data.withColumn('date', f.to_date('date')) \
        .withColumn('time', f.to_timestamp('time'))

udf_ = f.udf(lambda x: ast.literal_eval(x), t.ArrayType(t.StringType()))
data = data.withColumn('hashtags', udf_('hashtags')) \
    .withColumn('mentions', udf_('mentions'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[tweetid: string, username: string, content: string, reply: int, retweet: int, likes: int, date: date, time: timestamp, hashtags: string, mentions: string]

## Tokenizing tweet content

In [67]:
doc_assembler = DocumentAssembler().setInputCol("content").setOutputCol("document")

tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('tokenized')

normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') \
     .setLowercase(True)

lemmatizer = LemmatizerModel.pretrained() \
     .setInputCols(['normalized']) \
     .setOutputCol('lemmatized')

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('stopclean') \
     .setCaseSensitive(False)

stemmer = Stemmer().setInputCols(['stopclean']).setOutputCol('stem')

finisher = Finisher().setInputCols(['stem']).setOutputCols(['tokens'])

pipeline = Pipeline(
    stages=[doc_assembler, tokenizer, normalizer, lemmatizer, stopwords_cleaner, stemmer, finisher]
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]

In [68]:
model = pipeline.fit(data)
data_trans = model.transform(data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
data_trans['content', 'tokens'].show(1, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------+
|content                                                                                                                                                     |tokens                                                                                                       |
+------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------+
|“The timing is right for this place to rise.” Petition for 
@NJGov
 to reinstate the monarchy with 
@IAMQUEENLATIFAH
 #Newark  #NJ #affordablehousing #QUEEN|[time, right, place, rise, petit, n

## Find top words

In [73]:
wc = data_trans.withColumn('word', f.explode(f.col('tokens'))) \
  .groupBy('word').count().sort('count', ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
wc.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----+
|       word|count|
+-----------+-----+
|affordableh| 8435|
|       hous| 4021|
|     afford| 1975|
|        new| 1171|
|       home| 1133|
|     commun|  928|
|    develop|  851|
|      build|  833|
|         ne|  755|
|       help|  598|
|       citi|  565|
|       make|  550|
|       unit|  539|
|    project|  525|
|   homeless|  519|
|       work|  497|
|    support|  480|
|      learn|  456|
|        via|  455|
|       rent|  442|
+-----------+-----+
only showing top 20 rows

## Topic Modelling

In [75]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
cv = CountVectorizer(inputCol='tokens', outputCol='cv_features')
cv_model = cv.fit(data_trans)
data_cv = cv_model.transform(data_trans)

idf = IDF(inputCol='cv_features', outputCol='tf_idf_features')
idf_model = idf.fit(data_cv)
data_tfidf = idf_model.transform(data_cv)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
num_topics = 5
max_iter = 10
lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features')
lda_model = lda.fit(data_tfidf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
vocab = cv_model.vocabulary

def get_words(token_list):
    return [vocab[token_id] for token_id in token_list]

udf_to_words = f.udf(get_words, t.ArrayType(t.StringType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
nwords = 10

topics = lda_model.describeTopics(nwords) \
     .withColumn('topic_words', udf_to_words(f.col('termIndices')))

topics.select('topic', 'topic_words').show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+---------------------------------------------------------------------------------+
|topic|topic_words                                                                      |
+-----+---------------------------------------------------------------------------------+
|0    |[new, home, hous, on, afford, todai, support, team, celebr, thank]               |
|1    |[hous, rent, afford, ne, citi, build, increas, realest, market, big]             |
|2    |[hous, afford, home, help, homeless, commun, build, peopl, new, work]            |
|3    |[thank, help, commun, hous, righthous, park, grant, survei, coverag, particip]   |
|4    |[hous, afford, unit, multifamili, develop, haigroup, home, stori, construct, via]|
+-----+---------------------------------------------------------------------------------+